# モメンタム取引のためのオシレーター指標によるポジション判定 (Choppiness含む)

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import date, datetime, timedelta
import matplotlib.pyplot as plt
import talib
import warnings
import plotly.graph_objects as go
import mplfinance as mpf
import pickle
warnings.simplefilter(action="ignore", category=FutureWarning)

## ta-libにない関数の定義

In [2]:
# function Klinger Volume Oscillator (KVO)
def klinger_volume_oscillator(df, short_period=34, long_period=55, signal_period=13):
    df['High-Low'] = df['High'] - df['Low']
    df['High-Close'] = np.abs(df['High'] - df['Close'].shift(1))
    df['Low-Close'] = np.abs(df['Low'] - df['Close'].shift(1))
    df['TrueRange'] = df[['High-Low', 'High-Close', 'Low-Close']].max(axis=1)
    df['VolumeForce'] = np.where(df['Close'] > df['Close'].shift(1), df['Volume'], -df['Volume'])
    df['KVO'] = (df['VolumeForce'].rolling(window=short_period).sum() -
                 df['VolumeForce'].rolling(window=long_period).sum())
    df['KVO_Signal'] = df['KVO'].rolling(window=signal_period).mean()
    return df['KVO'], df['KVO_Signal']

# function Elder's Force Index (EFI)
def elder_force_index(df, period=2):
    df['EFI'] = (df['Close'] - df['Close'].shift(1)) * df['Volume']
    df['EFI_Smoothed'] = df['EFI'].rolling(window=period).mean()
    return df['EFI_Smoothed']

# function true_strength_index (TSI)
def true_strength_index(df, short_period=13, long_period=25):
    df['Change'] = df['Close'] - df['Close'].shift(1)
    df['Smoothed1'] = df['Change'].ewm(span=short_period, adjust=False).mean()
    df['Smoothed2'] = df['Smoothed1'].ewm(span=long_period, adjust=False).mean()
    df['AbsChange'] = abs(df['Change'])
    df['AbsSmoothed1'] = df['AbsChange'].ewm(span=short_period, adjust=False).mean()
    df['AbsSmoothed2'] = df['AbsSmoothed1'].ewm(span=long_period, adjust=False).mean()
    df['TSI'] = (df['Smoothed2'] / df['AbsSmoothed2']) * 100
    return df['TSI']

# オーサムオシレーター
def awesome_oscillator(df, wshort=5, wlong=34):
  # 中値
  df['med'] = (df['High']+df['Low']) / 2
  # オーサムオシレーター
  df['ao'] = df['med'].rolling(wshort).mean() - df['med'].rolling(wlong).mean()
  # 加速減速オシレーター
  df['ado'] = df['ao'] - df['ao'].rolling(wshort).mean()
  return df['ao'], df['ado']

# Choppiness Index
def choppiness_index(data, period=14):
    # Calculate the True Range
    high_low = data['High'] - data['Low']
    high_close = np.abs(data['High'] - data['Close'].shift(1))
    low_close = np.abs(data['Low'] - data['Close'].shift(1))
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    # Sum of the True Range over the given period
    atr = true_range.rolling(window=period).sum()
    # Calculate the difference between the highest high and lowest low over the period
    max_high = data['High'].rolling(window=period).max()
    min_low = data['Low'].rolling(window=period).min()
    # Calculate the Choppiness Index
    chop = 100 * np.log10(atr / (max_high - min_low)) / np.log10(period)
    return chop

## 銘柄リストの作成

In [3]:
# 銘柄コード辞書を読み込む
with open('JPX_code_dict.pickle', mode='rb') as f:
    jpx = pickle.load(f)

In [4]:
# チェック対象銘柄リストを読み込む
df01 = pd.read_excel('personal_invest.xlsx')
df02 = pd.read_excel('popular_invest.xlsx')
df1 = pd.concat([df01, df02], axis=0)

In [5]:
# 米国チェック対象銘柄リストを読み込む
df04 = pd.read_excel('personal_invest_us.xlsx')
df2 = df04[df04['flag'] == 1]
df2.tail()

,category,name,code,flag,source
36,軍需・宇宙,RTX Corp,RTX,1.0,みんかぶ人気テーマ
37,軍需・宇宙,ハネウェル・インターナショナル,HON,1.0,みんかぶ人気テーマ
38,原子力・航空機エンジン,GEエアロスペース,GE,1.0,みんかぶ人気テーマ
39,総合エネルギー,シェブロン,CVX,1.0,DMM人気銘柄
40,特殊化学製品,アルベマール,ALB,1.0,DMM人気銘柄


In [6]:
# 米国銘柄リストより仮辞書を作成して日本の辞書に結合する
df2.loc[:,'description'] = df2.copy().loc[:,'name'] + '：' + df2.copy().loc[:,'category']
df3 = df2.set_index('code')
usadd = df3.to_dict()['description']
jpx.update(usadd)

C:\Users\nicjp\AppData\Local\Temp\ipykernel_12504\510649325.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.loc[:,'description'] = df2.copy().loc[:,'name'] + '：' + df2.copy().loc[:,'category']


In [7]:
# コード番号からTickerリストを作成
df1['ticker'] = df1['code'].astype(str)+'.T'
list_ticker = df1['ticker'].tolist() + df2['code'].tolist()
print(list_ticker)

['9531.T', '9432.T', '7419.T', '8267.T', '2730.T', '2340.T', '9726.T', '9020.T', '8411.T', '4182.T', '8593.T', '2317.T', '4042.T', '8801.T', '8089.T', '9006.T', '6742.T', '2503.T', '7186.T', '7201.T', '8131.T', '5411.T', '4004.T', '2432.T', '6632.T', '2002.T', '5991.T', '1963.T', '7011.T', '7013.T', '2201.T', '2440.T', '1942.T', '6617.T', '6361.T', '5471.T', '9507.T', '9502.T', '9503.T', '9508.T', '9506.T', '9504.T', '1605.T', '2502.T', '2914.T', '3038.T', '3382.T', '4005.T', '4385.T', '4503.T', '4689.T', '4755.T', '5020.T', '5803.T', '6178.T', '6315.T', '6323.T', '6501.T', '6503.T', '6526.T', '6594.T', '6702.T', '6723.T', '6752.T', '6758.T', '6762.T', '6902.T', '6971.T', '7003.T', '7012.T', '7182.T', '7203.T', '7211.T', '7261.T', '7267.T', '7269.T', '7272.T', '7733.T', '8031.T', '8058.T', '8306.T', '8308.T', '8316.T', '8410.T', '8601.T', '8604.T', '8725.T', '8766.T', '8801.T', '9101.T', '9104.T', '9107.T', '9434.T', 'CHD', 'COST', 'HD', 'LMT', 'MKC', 'NKE', 'V', 'DUK', 'NEE', 'SO', 'J

## データの読み込みとシグナル判定

In [8]:
# 期間の指定
end_date = datetime.today()
start_date = end_date - timedelta(days=730)

In [9]:
# 各銘柄についてシグナル判定を行い結果を縦連結する
df0 = pd.DataFrame()
for symbol in list_ticker:
    # yahooサイトからデータをダウンロード
    data = yf.download(symbol, start_date, end_date)
    data.columns = [col[0] if isinstance(col, tuple) else col for col in data.columns]
    data['ticker'] = symbol
    df = data.reset_index()
    # 指標値の計算
    df['SMA_short'] = talib.SMA(df['Close'], timeperiod=5)
    df['SMA_long'] = talib.SMA(df['Close'], timeperiod=25)
    df['CMO'] = talib.CMO(df['Close'], timeperiod=14)
    df['KVO'], df['KVO_Signal'] = klinger_volume_oscillator(df)
    df['EFI'] = elder_force_index(df)
    df['TSI'] = true_strength_index(df)
    df['ARO'] = talib.AROONOSC(df['High'], df['Low'], timeperiod=25)
    df['MACD'], df['MACD_signal'], df['MACD_hist'] = talib.MACD(df['Close'], \
            fastperiod=12, slowperiod=26, signalperiod=9)
    df['AO'], df['ADO'] = awesome_oscillator(df)
    df['TRIX'] = talib.TRIX(df['Close'], timeperiod=9)
    df['CHOP'] = choppiness_index(df)
    # ポジションの判定
    df['SMA_buy'] = (df['SMA_short'] > df['SMA_long'])
    df['SMA_sell'] = (df['SMA_short'] < df['SMA_long'])
    df['CMO_buy'] = (df['CMO'] > 0)
    df['CMO_sell'] = (df['CMO'] < 0)
    df['KVO_buy'] = (df['KVO'] > 0)
    df['KVO_sell'] = (df['KVO'] < 0)
    df['EFI_buy'] = (df['EFI'] > 0)
    df['EFI_sell'] = (df['EFI'] < 0)
    df['TSI_buy'] = (df['TSI'] > 0)
    df['TSI_sell'] = (df['TSI'] < 0)
    df['ARO_buy'] = (df['ARO'] > 0)
    df['ARO_sell'] = (df['ARO'] < 0)
    df['MACD_buy'] = (df['MACD'] > df['MACD_signal'])
    df['MACD_sell'] = (df['MACD'] < df['MACD_signal'])
    df['AO_buy'] = (df['AO'] > 0)
    df['AO_sell'] = (df['AO'] < 0)
    df['TRIX_buy'] = (df['TRIX'] > 0)
    df['TRIX_sell'] = (df['TRIX'] < 0)
    df['CHOP_range'] = (df['CHOP'] > 61.8)
    df['CHOP_trend'] = (df['CHOP'] < 38.2)
    # 一銘柄のシグナル判定結果の最終行のみを元のデータに縦連結する
    df0 = pd.concat([df0,df.tail(1)],axis=0)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

## 結果の整理と表示

In [10]:
# コードから銘柄名を返す関数
def ticker_dict(df):
    return jpx.get(df)

In [11]:
# 銘柄名の付与
# 後で参照するためreset_index()によるインデックスのふり直しは必要
df_signal = df0.loc[:,['Date','ticker','Close',\
                       'SMA_buy','SMA_sell',\
                       'CMO_buy','CMO_sell','KVO_buy','KVO_sell',\
                       'EFI_buy','EFI_sell','TSI_buy','TSI_sell',\
                       'ARO_buy','ARO_sell','MACD_buy','MACD_sell',\
                       'AO_buy','AO_sell','TRIX_buy','TRIX_sell',\
                       'CHOP_range','CHOP_trend']].reset_index()
df_signal['ticker2'] = df_signal['ticker'].mask(df_signal['ticker'].str.endswith('.T'),df_signal['ticker'].str[:-2])
df_signal['cname'] = df_signal['ticker2'].map(ticker_dict)
df_signal.head()

,index,Date,ticker,Close,SMA_buy,SMA_sell,CMO_buy,CMO_sell,KVO_buy,KVO_sell,...,MACD_buy,MACD_sell,AO_buy,AO_sell,TRIX_buy,TRIX_sell,CHOP_range,CHOP_trend,ticker2,cname
0,490,2025-01-29,9531.T,4052.000000,False,True,False,True,False,True,...,False,True,False,True,False,True,False,False,9531,東京瓦斯
1,490,2025-01-29,9432.T,153.199997,False,True,False,True,False,True,...,True,False,False,True,False,True,False,False,9432,日本電信電話
2,490,2025-01-29,7419.T,2205.000000,False,True,True,False,False,True,...,True,False,False,True,False,True,False,False,7419,ノジマ
3,490,2025-01-29,8267.T,3722.000000,True,False,True,False,True,False,...,True,False,False,True,True,False,False,False,8267,イオン
4,490,2025-01-29,2730.T,1820.000000,True,False,True,False,False,True,...,True,False,True,False,True,False,False,False,2730,エディオン


In [12]:
# シグナルメッセージの付与
df_signal["signal_str"] = np.nan
df_signal["signal_total"] = np.nan
for irow in range(len(df_signal)):
    str_signal = ''
    total_signal = 0
    if df_signal.SMA_buy[irow]:
        str_signal += 'SMA+,'
        total_signal += 100
    if df_signal.SMA_sell[irow]:
        str_signal += 'SMA-,'
        total_signal -= 100
    if df_signal.CMO_buy[irow]:
        str_signal += 'CMO+,'
        total_signal += 100
    if df_signal.CMO_sell[irow]:
        str_signal += 'CMO-,'
        total_signal -= 100
    if df_signal.KVO_buy[irow]:
        str_signal += 'KVO+,'
        total_signal += 100
    if df_signal.KVO_sell[irow]:
        str_signal += 'KVO-,'
        total_signal -= 100
    if df_signal.EFI_buy[irow]:
        str_signal += 'EFI+,'
        total_signal += 100
    if df_signal.EFI_sell[irow]:
        str_signal += 'EFI-,'
        total_signal -= 100
    if df_signal.TSI_buy[irow]:
        str_signal += 'TSI+,'
        total_signal += 100
    if df_signal.TSI_sell[irow]:
        str_signal += 'TSI-,'
        total_signal -= 100
    if df_signal.ARO_buy[irow]:
        str_signal += 'ARO+,'
        total_signal += 100
    if df_signal.ARO_sell[irow]:
        str_signal += 'ARO-,'
        total_signal -= 100
    if df_signal.MACD_buy[irow]:
        str_signal += 'MACD+,'
        total_signal += 100
    if df_signal.MACD_sell[irow]:
        str_signal += 'MACD-,'
        total_signal -= 100
    if df_signal.AO_buy[irow]:
        str_signal += 'AO+,'
        total_signal += 100
    if df_signal.AO_sell[irow]:
        str_signal += 'AO-,'
        total_signal -= 100
    if df_signal.TRIX_buy[irow]:
        str_signal += 'TRIX+,'
        total_signal += 100
    if df_signal.TRIX_sell[irow]:
        str_signal += 'TRIX-,'
        total_signal -= 100
    if df_signal.CHOP_range[irow]:
        str_signal += 'CHOPr,'
    if df_signal.CHOP_trend[irow]:
        str_signal += 'CHOPt,'
        total_signal *= 10
    df_signal.loc[irow,"signal_str"] = str_signal
    df_signal.loc[irow,"signal_total"] = total_signal

In [13]:
# シグナル一覧の表示
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
df_tmp = df_signal[['Date', 'ticker', 'cname', 'signal_str', 'signal_total']]
df_table = df_tmp.sort_values(by='signal_total',ascending=False)
df_table

,Date,ticker,cname,signal_str,signal_total
88,2025-01-29,8801.T,三井不動産,"SMA+,CMO+,KVO+,EFI+,TSI+,ARO+,MACD+,AO+,TRIX+,CHOPt,",9000.0
13,2025-01-29,8801.T,三井不動産,"SMA+,CMO+,KVO+,EFI+,TSI+,ARO+,MACD+,AO+,TRIX+,CHOPt,",9000.0
50,2025-01-29,4689.T,ＬＩＮＥヤフー,"SMA+,CMO+,KVO+,EFI+,TSI+,ARO+,MACD+,AO+,TRIX+,CHOPt,",9000.0
122,2025-01-28,SBUX,スターバックス：飲食,"SMA+,CMO+,KVO+,EFI+,TSI+,ARO+,MACD+,AO+,TRIX+,CHOPt,",9000.0
128,2025-01-28,RTX,RTX Corp：軍需・宇宙,"SMA+,CMO+,KVO+,EFI+,TSI+,ARO+,MACD+,AO+,TRIX+,CHOPt,",9000.0
43,2025-01-29,2502.T,アサヒグループホールディングス,"SMA+,CMO+,KVO+,EFI+,TSI+,ARO+,MACD+,AO+,TRIX+,CHOPt,",9000.0
106,2025-01-28,UNP,ユニオンパシフィック：鉄道,"SMA+,CMO+,KVO+,EFI+,TSI+,ARO+,MACD+,AO+,TRIX+,CHOPt,",9000.0
85,2025-01-29,8604.T,野村ホールディングス,"SMA+,CMO+,KVO-,EFI+,TSI+,ARO+,MACD+,AO+,TRIX+,CHOPt,",7000.0
95,2025-01-28,HD,ホームデポ：ホームセンター,"SMA+,CMO+,KVO-,EFI+,TSI+,ARO+,MACD+,AO+,TRIX+,CHOPt,",7000.0
48,2025-01-29,4385.T,メルカリ,"SMA+,CMO+,KVO-,EFI+,TSI+,ARO+,MACD+,AO+,TRIX+,CHOPt,",7000.0
